<a href="https://colab.research.google.com/github/LuaraMarino/Trabalhinhos/blob/main/MachineLearning_RegressaoLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Nome: Luara Maria Marino
#RM: 89375

FIAP - Machine Learning & Modelling

# Aula 06 - Regressão Linear

Vamos passar pelo pipeline de ciência de dados, implementando os passos em python e realizando uma Regressão Linear como modelo preditivo.

## Pipeline de ciência de dados

Relembrando as etapas do processo de ciência de dados:

![image.png](attachment:image.png)

### Entendimento do problema

É possível prever a altura dos filhos baseado nas alturas dos pais? 


### Obtenção dos dados 

Este conjunto de dados lista as observações individuais de 934 crianças em 205 famílias nas quais Galton (1886) baseou sua tabulação cruzada, mostrando a relação entre as alturas dos pais e de seus filhos. 

Dados baixados [deste link](https://raw.githubusercontent.com/data-8/materials-fa17/master/lec/galton.csv)

Dicionário de dados retirado [daqui](https://vincentarelbundock.github.io/Rdatasets/doc/HistData/GaltonFamilies.html)

Os dados são: 
							
- `family`: ID de família, um fator com níveis 001-204
- `father`: altura do pai
- `mother`: altura da mãe
- `midparentHeight`: altura parental média, calculada como (pai + 1,08 * mãe) / 2
- `children`: número de filhos nesta família
- `childNum`: número desta criança dentro da família. As crianças são listadas em ordem decrescente de altura para meninos, seguidos por meninas
- `gender`: gênero infantil, fator com níveis feminino e masculino
- `childHeight`: altura da criança

**OBSERVAÇÃO**: os dados das alturas estão em `inches`. Para transformar em `cm`, podemos usar a fórmula: "1 inch =
2.54 centimeters"

In [ ]:
# Imports necessários 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
df = pd.read_csv(f"galton.csv")
df.head(5)

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
0,1,78.5,67.0,75.43,4,1,male,73.2
1,1,78.5,67.0,75.43,4,2,female,69.2
2,1,78.5,67.0,75.43,4,3,female,69.0
3,1,78.5,67.0,75.43,4,4,female,69.0
4,2,75.5,66.5,73.66,4,1,male,73.5


In [ ]:
# Quantos dados? 
df.shape

(934, 8)

In [ ]:
# Quais os tipos? Dados faltantes? 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   family           934 non-null    object 
 1   father           934 non-null    float64
 2   mother           934 non-null    float64
 3   midparentHeight  934 non-null    float64
 4   children         934 non-null    int64  
 5   childNum         934 non-null    int64  
 6   gender           934 non-null    object 
 7   childHeight      934 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 58.5+ KB


In [ ]:
df['children'].unique()

array([ 4,  2,  5,  6,  1,  3,  8,  9,  7, 11, 10, 15])

In [ ]:
df.nunique()

family             205
father              35
mother              29
midparentHeight    140
children            12
childNum            15
gender               2
childHeight         67
dtype: int64

In [ ]:
df[df["gender"] == "male" and df["childHeight"] > 78 ]

ValueError: ignored

In [ ]:
# Alguma info estatística útil? 
df.describe()

,father,mother,midparentHeight,children,childNum,childHeight
count,934.000000,934.000000,934.000000,934.000000,934.000000,934.000000
mean,69.197109,64.089293,69.206773,6.171306,3.585653,66.745931
std,2.476479,2.290886,1.802370,2.729025,2.361410,3.579251
min,62.000000,58.000000,64.400000,1.000000,1.000000,56.000000
25%,68.000000,63.000000,68.140000,4.000000,2.000000,64.000000
50%,69.000000,64.000000,69.248000,6.000000,3.000000,66.500000
75%,71.000000,65.875000,70.140000,8.000000,5.000000,69.700000
max,78.500000,70.500000,75.430000,15.000000,15.000000,79.000000


### Limpeza dos dados

In [ ]:
# Removendo duplicatas
df.drop_duplicates(inplace=True)

In [ ]:
# Removendo dados faltantes
df.dropna(inplace=True)

Vamos trabalhar com dados em **cm**? Não somos obrigados. 

In [ ]:
# Funcao

In [ ]:
def inch2m(inch):
    return inch * 2.54 / 100

In [ ]:
inch2m(78.5)

1.9939000000000002

In [ ]:
df["father"] = df["father"].apply(inch2m)

In [ ]:
df.head(5)

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
0,1,1.9939,67.0,75.43,4,1,male,73.2
1,1,1.9939,67.0,75.43,4,2,female,69.2
2,1,1.9939,67.0,75.43,4,3,female,69.0
3,1,1.9939,67.0,75.43,4,4,female,69.0
4,2,1.9177,66.5,73.66,4,1,male,73.5


Agora faça o mesmo para as demais colunas de alturas!

In [ ]:
colunas = ["mother", "midparentHeight", "childHeight"]

In [ ]:
df[colunas] = df[colunas].apply(inch2m)

Sabemos que os modelos só trabalham com números, então não podemos deixar a coluna `gender` como está! Precisamos transformá-la!

In [ ]:
df["gender"] = df["gender"].apply(lambda x: 0 if x == "male" else 1)

In [ ]:
df.head(5)

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
0,1,1.9939,1.7018,1.915922,4,1,0,1.85928
1,1,1.9939,1.7018,1.915922,4,2,1,1.75768
2,1,1.9939,1.7018,1.915922,4,3,1,1.75260
3,1,1.9939,1.7018,1.915922,4,4,1,1.75260
4,2,1.9177,1.6891,1.870964,4,1,0,1.86690


In [ ]:
# Removendo colunas que não serão úteis
df2 = df.drop(columns = ["family", "children", "childNum"])
df2.head(5)

,father,mother,midparentHeight,gender,childHeight
0,1.9939,1.7018,1.915922,0,1.85928
1,1.9939,1.7018,1.915922,1,1.75768
2,1.9939,1.7018,1.915922,1,1.75260
3,1.9939,1.7018,1.915922,1,1.75260
4,1.9177,1.6891,1.870964,0,1.86690


### Modelagem

In [ ]:
# Separação de DADOS e LABEL
X = df2.drop(columns = ["childHeight"])
Y = df2["childHeight"]

In [ ]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.6, random_state=42)

In [ ]:
# Treinando o modelo
modelo = LinearRegression()
modelo.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Fazendo as predições
Y_pred = modelo.predict(X_test)

### Avaliação

In [ ]:
# Erro quadrático médio
mean_squared_error(Y_test, Y_pred)

0.0031376204881859763

In [ ]:
# Erro absoluto médio
mean_absolute_error(Y_test, Y_pred)

0.044239311623267594

In [ ]:
# R²
r2_score(Y_test, Y_pred)

0.6364702607871328

### Interpretando os coeficientes encontrados!

Vamos dar uma olhada nos coeficientes encontrados para cada característica:

In [ ]:
modelo.intercept_

0.5549619417047795

In [ ]:
modelo.coef_

array([-6.39888541e+12, -6.91079624e+12,  1.27977708e+13, -1.32720947e-01])

In [ ]:
df2.columns

Index(['father', 'mother', 'midparentHeight', 'gender', 'childHeight'], dtype='object')

In [ ]:
pd.DataFrame({"feature": df2.columns[:-1], "coeficientes": modelo.coef_}).sort_values(by="coeficientes", ascending=False)

,feature,coeficientes
2,midparentHeight,1.279777e+13
3,gender,-1.327209e-01
0,father,-6.398885e+12
1,mother,-6.910796e+12


In [ ]:
modelo.intercept_

0.5549619417047795

In [ ]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [ ]:
# Treinando o modelo
modelo = LinearRegression()
modelo.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Fazendo as predições
Y_pred = modelo.predict(X_test)

In [ ]:
# Erro quadrático médio
mean_squared_error(Y_test, Y_pred)

0.003151716179372416

In [ ]:
# Erro absoluto médio
mean_absolute_error(Y_test, Y_pred)

0.04354312703914847

In [ ]:
# R²
r2_score(Y_test, Y_pred)

0.6288190815324056

In [ ]:
modelo.intercept_

0.5580062387963236

In [ ]:
modelo.coef_

array([-2.50725114e+13, -2.70783123e+13,  5.01450228e+13, -1.42578125e-01])

In [ ]:
df2.columns

Index(['father', 'mother', 'midparentHeight', 'gender', 'childHeight'], dtype='object')

In [ ]:
pd.DataFrame({"feature": df2.columns[:-1], "coeficientes": modelo.coef_}).sort_values(by="coeficientes", ascending=False)

,feature,coeficientes
2,midparentHeight,5.014502e+13
3,gender,-1.425781e-01
0,father,-2.507251e+13
1,mother,-2.707831e+13


In [ ]:

modelo.intercept_

0.5580062387963236

In [ ]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
# Treinando o modelo
modelo = LinearRegression()
modelo.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Fazendo as predições
Y_pred = modelo.predict(X_test)

In [ ]:
# Erro quadrático médio
mean_squared_error(Y_test, Y_pred)

0.0027017304306377068

In [ ]:
# Erro absoluto médio
mean_absolute_error(Y_test, Y_pred)

0.04223601664992338

In [ ]:
# R²
r2_score(Y_test, Y_pred)

0.6325596511413618

In [ ]:
modelo.intercept_

0.554812428497583

In [ ]:
modelo.coef_

array([-5.22928577e+12, -5.64762863e+12,  1.04585715e+13, -1.32446289e-01])

In [ ]:
df2.columns

Index(['father', 'mother', 'midparentHeight', 'gender', 'childHeight'], dtype='object')

In [ ]:
pd.DataFrame({"feature": df2.columns[:-1], "coeficientes": modelo.coef_}).sort_values(by="coeficientes", ascending=False)

,feature,coeficientes
2,midparentHeight,1.045857e+13
3,gender,-1.324463e-01
0,father,-5.229286e+12
1,mother,-5.647629e+12


In [ ]:
modelo.intercept_

0.554812428497583

In [ ]:
#Resultados dos modelos
#Train size 0.6 -> 0.5549619417047795
#Train size 0.4 -> 0.5580062387963236
#Train size 0.1 -> 0.554812428497583